In [2]:
#start
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [2]:
date_columns = ['expiration_date', 'registration_init_time']

train_data = pd.read_csv('E:/KKBOX/train.csv')
test_data = pd.read_csv('E:/KKBOX/test.csv', index_col=0)
item_data = pd.read_csv('E:/KKBOX/songs.csv')
user_data = pd.read_csv('E:/KKBOX/members.csv', parse_dates=date_columns)

all_data_org = pd.concat([train_data, test_data])
all_data_org = all_data_org.merge(item_data, on='song_id', how='left')
all_data_org = all_data_org.merge(user_data, on='msno', how='left')
#print(all_data_org[0:6])
#all_data_org.to_csv('E:/KKBOX/KKBOXNo3/all_data_org.csv')

c:\users\tangx\appdata\local\programs\python\python36\lib\site-packages\numpy\lib\arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
enc = LabelEncoder()

for col in [
    'msno', 'song_id', 'source_screen_name', 
    'source_system_tab', 'source_type', 
    'artist_name', 'composer', 'lyricist', 'gender'
]:
    all_data_org[col] = enc.fit_transform(all_data_org[col].fillna('nan')
    
for col in ['language', 'city', 'registered_via']:
    all_data_org[col] = enc.fit_transform(all_data_org[col].fillna(-2)) 

In [4]:
all_data_org['time'] = all_data_org.index / len(all_data_org)

n = len(train_data)
train_data = all_data_org[:n]
test_data = all_data_org[n:]
print(n)

#train_data.to_hdf('data/train_data.hdf', key='wsdm')
#test_data.to_hdf('data/test_data.hdf', key='wsdm')
#train_data.to_hdf('E:/KKBOX/wsdm-kkboxmusicrec-py/train_data.hdf',key='candy')
#test_data.to_hdf('E:/KKBOX/wsdm-kkboxmusicrec-py/test_data.hdf',key='candy')

7377418


In [3]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression

In [4]:
from itertools import combinations

import scipy.sparse as sp
from scipy.sparse import coo_matrix
from lightfm import LightFM

c:\users\tangx\appdata\local\programs\python\python36\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn('LightFM was compiled without OpenMP support. '


In [5]:
date_cols = ['expiration_date', 'registration_init_time']

train_data = pd.read_hdf('E:/KKBOX/wsdm-kkboxmusicrec-py/train_data.hdf', parse_dates=date_cols)
test_data = pd.read_hdf('E:/KKBOX/wsdm-kkboxmusicrec-py/test_data.hdf', parse_dates=date_cols)

In [8]:
all_data=pd.read_csv('E:/KKBOX/wsdm-kkboxmusicrec-py/all_data.csv')
#all_data = pd.concat([train_data, test_data])

In [9]:
df_test = test_data
df_history_test = train_data

df_trains = []
df_history_trains = []

n = len(test_data)
shift = int(0.05*len(train_data))

for i in range(2):
    m = -i*shift
    if m == 0:
        m = None
    df_trains.append(train_data[-(n + i*shift):m])
    df_history_trains.append(train_data[:-(n + i*shift)])
not_categorical_columns = [
    'target', 
    'song_length', 
    'registration_init_time', 
    'expiration_date', 
    'time', 
    'bd',
]
categorical_columns = all_data.columns.difference(not_categorical_columns)

orders = {}

for col in categorical_columns:
    orders[col] = 10 ** (int(np.log(all_data[col].max() + 1) / np.log(10)) + 1)

orders

{'Unnamed: 0': 10000000,
 'artist_name': 100000,
 'city': 100,
 'composer': 100000,
 'gender': 10,
 'genre_ids': 1000,
 'language': 100,
 'lyricist': 100000,
 'msno': 100000,
 'registered_via': 10,
 'song_id': 1000000,
 'source_screen_name': 100,
 'source_system_tab': 10,
 'source_type': 100}

In [69]:
print(df_history_trains[0][800:806])

      msno  song_id  source_screen_name  source_system_tab  source_type  \
800  34303   373882                   8                  3            4   
801  34303    94001                   8                  3            4   
802  14831   221923                   8                  3            3   
803  29979   389392                   8                  3            4   
804  34303    90017                   8                  3            4   
805  34303   108836                   8                  3            4   

     target  song_length  genre_ids  artist_name  composer  lyricist  \
800     1.0     365165.0        371        43379     32131     10660   
801     1.0     227253.0        371        44392     29339     24946   
802     1.0     264150.0        371        42671     79748     30743   
803     1.0     243356.0        371        42525     74276     26024   
804     1.0     305946.0        371        43379     32131     10660   
805     1.0     261387.0        371       

In [22]:
def get_group(df, cols): #获取二元组或三元组的唯一编号
    group = df[cols[0]].copy()
    for col in cols[1:]:
        group = group * orders[col] + df[col]
  
    return group

get_group(df_trains[0],list(['msno','source_screen_name']))

4820628    1819822
4820629    1819817
4820630    3427800
4820631    3427800
4820632    3427800
4820633    3427800
4820634    3427800
4820635    3427800
4820636    3427808
4820637    3427808
4820638    3427800
4820639    2638408
4820640     798708
4820641     798708
4820642     798708
4820643     798708
4820644     798708
4820645     798708
4820646     798708
4820647     798708
4820648     798708
4820649     798708
4820650     798708
4820651     798708
4820652     798708
4820653     798708
4820654     798708
4820655     135304
4820656      96008
4820657     800311
            ...   
7377388    2568608
7377389    2568608
7377390    3006908
7377391    1852008
7377392     868500
7377393     868500
7377394    2737708
7377395    2737708
7377396    2737708
7377397     723408
7377398    1521008
7377399    1521008
7377400    1521008
7377401    1910908
7377402     761122
7377403     761122
7377404     761122
7377405    2943108
7377406    3093811
7377407    1448608
7377408    2854017
7377409    2

In [52]:
def mean(df_history, df, cols):
    
    group = get_group(df, cols)
    group_history = get_group(df_history, cols)
    #print(group_history)
    mean_map = df_history.groupby(group_history).target.mean() #把匹配的两列相同的属性算作一组，每一组计算均值
    #print(df_history.groupby(group_history).mean())
    #print(mean_map)
    return group.map(mean_map).fillna(-1) 

mean(df_history_trains[0],df_trains[0],list(['msno','source_screen_name','source_type']))

4820628    0.000000
4820629    0.000000
4820630    0.476190
4820631    0.476190
4820632    0.476190
4820633    0.476190
4820634    0.476190
4820635    0.476190
4820636    0.151515
4820637    0.151515
4820638    0.476190
4820639    0.890244
4820640    0.931818
4820641    0.931818
4820642    0.931818
4820643    0.931818
4820644    0.931818
4820645    0.931818
4820646    0.931818
4820647    0.931818
4820648    0.931818
4820649    0.931818
4820650    0.931818
4820651    0.931818
4820652    0.931818
4820653    0.931818
4820654    0.931818
4820655   -1.000000
4820656    0.575342
4820657    0.352941
             ...   
7377388   -1.000000
7377389   -1.000000
7377390    0.812500
7377391    0.851351
7377392    0.250000
7377393    0.250000
7377394    0.285714
7377395    0.285714
7377396    0.285714
7377397    0.824295
7377398    0.616667
7377399    0.616667
7377400    0.616667
7377401   -1.000000
7377402   -1.000000
7377403   -1.000000
7377404   -1.000000
7377405    0.423077
7377406    0.866667


In [47]:
group = get_group(df_trains[0], list(['msno','source_screen_name','source_type']))
group_history = get_group(df_history_trains[0], list(['msno','source_screen_name','source_type']))
#print(group_history)
mean_map = df_history_trains[0].groupby(group_history).target.mean()

In [ ]:
def get_group(df, cols): #获取二元组或三元组
    
    group = df[cols[0]].copy()
    for col in cols[1:]:
        group = group * orders[col] + df[col]
        
    return group


def mean(df_history, df, cols): #
    
    group = get_group(df, cols)
    group_history = get_group(df_history, cols)
    
    mean_map = df_history.groupby(group_history).target.mean()
    
    return group.map(mean_map).fillna(-1)


def count(df_history, df, cols):
    
    group = get_group(df, cols)
    group_all = get_group(all_data, cols)
    
    count_map = group_all.value_counts()
    
    return group.map(count_map).fillna(0)

In [ ]:
def col_name(cols, func):
    return '_'.join(cols) + '_' + func.__name__


def create_features(df, df_history):
    
    X = pd.DataFrame()
    
    for num_col in [1, 2]:
        for cols in combinations(categorical_columns, num_col):
            for func in [
                mean, 
                count, 
                time_to_next_heard, 
                count_from_future,
                last_time_diff, 
                count_from_past
            ]:
                X[col_name(cols, func)] = func(df_history, df, list(cols))
    
    for cols in combinations(categorical_columns, 3):
        for func in [mean, count]:
            X[col_name(cols, func)] = func(df_history, df, list(cols))
        if 'msno' in cols:
            for func in [time_to_next_heard, last_time_diff, count_from_past]:
                X[col_name(cols, func)] = func(df_history, df, list(cols))

    for cols in [
         ['msno'], 
         ['msno', 'source_type'], 
         ['msno', 'genre_ids'], 
         ['msno', 'artist_name'], 
         ['msno', 'composer'], 
         ['msno', 'language'], 
         ['song_id']
     ]:
        X[col_name(cols, regression)] = regression(df_history, df, cols)

    for cols in [
        ['msno'], 
        ['msno', 'genre_ids'],
        ['msno', 'composer'], 
        ['msno', 'language'], 
        ['msno','artist_name']
    ]:
        X[col_name(cols, time_from_prev_heard)] = time_from_prev_heard(df_history, df, cols)

    for col in ['song_length', 'bd']:
        X[col] = df[col]
        
    for col in ['expiration_date', 'registration_init_time']:
        X[col] = df[col].apply(lambda x: x.toordinal())
        
    X['part_song_listened'] = df['song_length'] / X['msno_time_to_next_heard'] 
    X['time_from_test_period'] = np.arange(len(df))
    X['part_of_unique_song'] = part_of_unique_song(df)
    
    X['matrix_factorization'] = matrix_factorization(df, df_history)
    
    for i in [500000, 2000000]:
        for cols in [
             ['msno'], 
             ['msno', 'source_type'], 
             ['msno', 'genre_ids'], 
             ['msno', 'artist_name'], 
             ['msno', 'composer'], 
             ['msno', 'language'], 
             ['song_id']
        ]:
            X[col_name(cols, mean) + str(i)] = mean(df_history[-i:], df, cols)
    
    return X